In [43]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split

import json
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt

In [21]:
with open('json/2019_pass_or_fail.json', encoding='UTF8') as file:
    dataset = json.load(file)

In [22]:
# JSON 데이터를 pandas DataFrame으로 변환
df = pd.DataFrame(dataset)

# "최저 지원 유무" 열 삭제
df_modify = df.drop(columns=['최저적용유무', '최초지원결과'])

# 빈 값이 있는 행 제거
df_modify = df_modify.replace('', pd.NA).dropna()

# 결과를 다시 JSON 형식으로 변환
modified_data = df_modify.to_dict(orient='records')

In [23]:
def hash_to_int(value, range_start=1, range_end=500):
    hashed_value = hash(value)
    mapped_value = (hashed_value % (range_end - range_start + 1)) + range_start
    return mapped_value

In [24]:
# 숫자로 변환하여 NumPy 배열 생성
numeric_data = np.array([
    [
        data['전과목(평균)'],
        data['국영수사(평균)'],
        data['국영수과(평균)'],
        data['국영수탐(백분위)'],
        data['국영수탐(등급)']
    ] 
    for data in modified_data
])

# 문자열 데이터를 numpy 배열로 변환 (hashing trick 적용) --> 수정 필요
string_data = np.array([
    [
        hash_to_int(data['지역']),
        hash_to_int(data['계열']),
        hash_to_int(data['대학명']),
        hash_to_int(data['학과명']),
        hash_to_int(data['전형유형']),
        hash_to_int(data['전형명'])
    ]
    for data in modified_data
])

# 결과값을 정수로 매핑 ("최종지원결과"를 1 또는 0으로 변환)
result = np.array([data['최종지원결과'] for data in modified_data])

In [25]:
print(numeric_data[0:5])
print(string_data[0:5])
print(result[0:5])

[['1.32' '1.31' '1.22' '252.5' '2.67']
 ['1.47' '1.51' '1.29' '281.5' '1.67']
 ['1.3' '1.3' '1.3' '225.5' '3.67']
 ['1.39' '1.44' '1.33' '224' '3.33']
 ['1.59' '1.41' '1.36' '263.5' '2.17']]
[[341 497  78 160 416 273]
 [341 497  78 160 416 273]
 [341 497  78 160 416 273]
 [341 497  78 160 416 273]
 [341 497  78 160 416 273]]
['합' '합' '합' '합' '불']


In [26]:
# 두 데이터를 수평으로 결합
numeric_data = np.array(numeric_data).astype(float)

print(numeric_data[0:2])
print(string_data[0:2])

# 두 배열을 수평으로 결합
combined_data = np.concatenate((string_data, numeric_data), axis=1, )
combined_data = np.array(combined_data, dtype=np.float64)
print(combined_data[0:2])

# 2차원 배열의 각 행에 대해 문자열을 정수로 변환
X = np.array(combined_data).astype(float)

# y값 문자열에 대해 원-핫 인코딩
y = np.array(result)
e = LabelEncoder()
e.fit(y)
Y = e.transform(y)

print(X[0:2])
print(Y[0:2])

[[  1.32   1.31   1.22 252.5    2.67]
 [  1.47   1.51   1.29 281.5    1.67]]
[[341 497  78 160 416 273]
 [341 497  78 160 416 273]]
[[341.   497.    78.   160.   416.   273.     1.32   1.31   1.22 252.5
    2.67]
 [341.   497.    78.   160.   416.   273.     1.47   1.51   1.29 281.5
    1.67]]
[[341.   497.    78.   160.   416.   273.     1.32   1.31   1.22 252.5
    2.67]
 [341.   497.    78.   160.   416.   273.     1.47   1.51   1.29 281.5
    1.67]]
[1 1]


In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

In [46]:
# 모델 생성
model = Sequential()
model.add(Dense(30, input_dim=11, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
# 마지막 레이어는 예측결과(sigmoid -> 0~1확률)
model.add(Dense(1, activation='sigmoid')) 

# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 모델 실행
history = model.fit(np.array(X_train), np.array(Y_train), epochs=100, batch_size=128, validation_batch_size=0.2)

score = model.evaluate(X_test, Y_test)
print("Test Accuracy: ", score[1])

Epoch 1/100
816/816 [==============================] - 2s 1ms/step - loss: 0.7958 - accuracy: 0.8330
Epoch 2/100
816/816 [==============================] - 1s 1ms/step - loss: 0.3984 - accuracy: 0.8571
Epoch 3/100
816/816 [==============================] - 1s 1ms/step - loss: 0.3813 - accuracy: 0.8591
Epoch 4/100
816/816 [==============================] - 1s 1ms/step - loss: 0.3747 - accuracy: 0.8584
Epoch 5/100
816/816 [==============================] - 1s 1ms/step - loss: 0.3715 - accuracy: 0.8576
Epoch 6/100
816/816 [==============================] - 1s 1ms/step - loss: 0.3698 - accuracy: 0.8575
Epoch 7/100
816/816 [==============================] - 1s 1ms/step - loss: 0.3687 - accuracy: 0.8579
Epoch 8/100
816/816 [==============================] - 1s 1ms/step - loss: 0.3703 - accuracy: 0.8574
Epoch 9/100
816/816 [==============================] - 1s 1ms/step - loss: 0.3656 - accuracy: 0.8585
Epoch 10/100
816/816 [==============================] - 1s 994us/step - loss: 0.3659 - accu

In [47]:
hist_df = pd.DataFrame(history.history)
print(hist_df)

        loss  accuracy
0   0.795819  0.832963
1   0.398440  0.857131
2   0.381335  0.859084
3   0.374712  0.858433
4   0.371464  0.857561
..       ...       ...
95  0.338948  0.860798
96  0.339116  0.861468
97  0.339009  0.861152
98  0.339504  0.860874
99  0.338643  0.861401

[100 rows x 2 columns]


In [44]:
hist_df = pd.DataFrame(history.history)
hist_df

# 테스트셋 오차
y_vloss = hist_df['val_loss']
# 학습셋 오차
y_loss = hist_df['loss']

# 테스트셋 오차 -> 빨간색, 학습셋 오차 -> 파란색
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, "o", c="red", markersize=2, label="Testset_loss")
plt.plot(x_len, y_loss, "o", c="blue", markersize=2, label="Trainset_loss")

plt.legend(loc="upper right")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()


KeyError: 'val_loss'